In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import zero_one_loss
import random
import pickle
import os
from dask import compute
import dask.bag as db
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from collections import Counter
from random import randrange

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier

Module 3

Intervally increase the biased value to show when certain features are finally not selected, get that threshold for each important sensors

In [2]:
# Get feature importance from initial model

In [3]:
# weather = 'TN_Knoxville'

# # initialize result df
# cv_error_df = pd.DataFrame([])

# # self-defined fault types
# selected_fault_types = ['air_handling_unit_fan_motor_degradation',
#                         'biased_economizer_sensor_mixed_t',
#                         'duct_fouling',
#                         'economizer_opening_stuck',
#                         'hvac_setback_error_delayed_onset',
#                         'hvac_setback_error_no_overnight_setback',
#                         'hvac_setback_error_early_termination',
#                         'improper_time_delay_setting_in_occupancy_sensors',
#                         'lighting_setback_error_delayed_onset',
#                         'lighting_setback_error_no_overnight_setback',
#                         'lighting_setback_error_early_termination',
#                         'return_air_duct_leakages',
#                         'supply_air_duct_leakages',
#                         'thermostat_bias'
#                        ]    

# print(f'Processing: {weather}...')

# # read metadata file
# prefixed = [filename for filename in os.listdir(f'data/{weather}/{weather}/') if 'sensors' not in filename]
# if len(prefixed) == 1:
#     meta_data_df = pd.read_csv(f'data/{weather}/{weather}/' + prefixed[0])
# else:
#     raise Exception("Something wrong with finding the meta data file in the data folder. Make sure the original data folder are used for data processing")

# # define the fault_type_list. Baseline is removed from the fault type
# #     fault_type_list = meta_data_df.fault_type.unique().tolist()
# #     fault_type_list.remove('baseline')
# fault_type_list = selected_fault_types

# inputs = pd.DataFrame([])
# output = pd.DataFrame([])

# for fault_type in fault_type_list:

#     #print(f' Processing: {fault_type}...')

#     # first focus on one weather file and one fault type
#     ids = meta_data_df.loc[meta_data_df.fault_type == fault_type].id.tolist()
#     intensities = meta_data_df.loc[meta_data_df.fault_type == fault_type].fault_intensity.tolist()

#     # non-fault IDs
#     ids_none_fault = meta_data_df.loc[meta_data_df.fault_type == 'baseline'].id.tolist()

#     # load data and make it from 15minute interval to hourly/daily data
#     fault_data_df = pd.DataFrame([])
#     for id_n, intensity in zip(ids, intensities):
#         temp_df = pd.read_csv(f'data/{weather}/{weather}/{id_n}_sensors.csv')
#         temp_df = temp_df.iloc[:,1:]
#         temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
#         #temp_df.loc[:,'label'] = intensity
#         temp_df.loc[:,'label'] = fault_type
#         fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)

#     # dealing with non fault data
#     temp_df = pd.read_csv(f'data/{weather}/{weather}/{ids_none_fault[0]}_sensors.csv')
#     temp_df = temp_df.iloc[:,1:]
#     temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
#     temp_df.loc[:,'label'] = 'none'
#     fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)
#     fault_data_df = fault_data_df.reset_index(drop = True)

#     inputs = pd.concat([inputs, fault_data_df.iloc[:,0:-9]], axis = 0, ignore_index = True)
#     output = pd.concat([output, fault_data_df.iloc[:,-1]], axis = 0, ignore_index = True)

# print(f' Processing: inputs/output data ready for {weather} ...')

# # cross-validation
# cv = KFold(n_splits=5, shuffle=True, random_state = 42)

# # Iterate through CV splits
# results = []
# important_features = []
# i = 0
# for train_index, test_index in cv.split(inputs):
#     X_train, X_test = inputs.iloc[train_index], inputs.iloc[test_index]
#     y_train, y_test = output.iloc[train_index], output.iloc[test_index]
#     # Fit the model on training data
#     regr = RandomForestClassifier(n_estimators = 2, random_state=42)
#     regr.fit(X_train, y_train.iloc[:,0])
#     # save model
#     filename = f'model/saved_model_{i}_{weather}.sav'
#     pickle.dump(regr, open(filename, 'wb'))
#     i += 1
#     # feature importance
#     feature_importance_temp = pd.DataFrame([])
#     feature_importance_temp.loc[:,'sensor_name'] = inputs.columns
#     feature_importance_temp.loc[:,'importance'] = regr.feature_importances_
#     important_features += feature_importance_temp.sort_values(
#         by=['importance'], ascending = False).sensor_name[0:20].tolist()
#     # Generate predictions on the test data and collect
#     y_test_predicted = regr.predict(X_test)
#     testing_error = zero_one_loss(y_test, y_test_predicted, normalize=True)
#     results += [testing_error]

# CV_error = sum(results)/len(results)
# cv_error_df.loc[:,weather + 'Baseline'] = [weather, 'all_sensors', 'all_inaccuracy', CV_error]
# important_features = list(set(important_features))

# # Sensor precision degradation
# results = []
# important_features_degradation = []
# print(f'  Processing: Degradation, {weather}...')
# for sensor in important_features:
#     i = 0
#     for train_index, test_index in cv.split(inputs):
#         X_train, X_test = inputs.iloc[train_index].copy(), inputs.iloc[test_index].copy()
#         y_train, y_test = output.iloc[train_index].copy(), output.iloc[test_index].copy()
#         # degradation
#         random_list = []
#         for j in range(len(X_test)):
#             random_list += [random.uniform(- X_test.loc[:,sensor].mean() * 0.05, X_test.loc[:,sensor].mean() * 0.05)]
#         X_test.loc[:,sensor] = X_test.loc[:,sensor] + random_list
#         # load saved pickle file
#         filename = f'model/saved_model_{i}_{weather}.sav'
#         regr = pickle.load(open(filename, 'rb'))
#         i += 1
#         # Generate predictions on the test data and collect
#         y_test_predicted = regr.predict(X_test)
#         testing_error = zero_one_loss(y_test, y_test_predicted, normalize=True)
#         results += [testing_error]
#         feature_importance_temp = pd.DataFrame([])
#         feature_importance_temp.loc[:,'sensor_name'] = inputs.columns
#         feature_importance_temp.loc[:,'importance'] = regr.feature_importances_
#         important_features_degradation += feature_importance_temp.sort_values(by=['importance'], ascending = False).sensor_name[0:20].tolist()
    
#     important_features_degradation = list(set(important_features_degradation))
    
#     if sensor in important_features_degradation:
#         continue
#     else:
#         break

#     CV_error = sum(results)/len(results)
#     cv_error_df.loc[:,weather + sensor + 'degradation'] = [weather, sensor, 'degradation', CV_error]


In [4]:
# all_fault_types = ['air_handling_unit_fan_motor_degradation',
#                         'biased_economizer_sensor_mixed_t',
#                         'duct_fouling',
#                         'economizer_opening_stuck',
#                         'hvac_setback_error_delayed_onset',
#                         'hvac_setback_error_no_overnight_setback',
#                         'hvac_setback_error_early_termination',
#                         'improper_time_delay_setting_in_occupancy_sensors',
#                         'lighting_setback_error_delayed_onset',
#                         'lighting_setback_error_no_overnight_setback',
#                         'lighting_setback_error_early_termination',
#                         'return_air_duct_leakages',
#                         'supply_air_duct_leakages',
#                         'thermostat_bias',]

# final_list = []

# for weather in ['AK_Fairbanks', 'FL_Miami', 'KY_Louisville', 'MN_Duluth', 'SAU_Riyadh', 'TN_Knoxville', 'VA_Richmond']:
#     print(weather)

#     for fault_type_n in all_fault_types:
#         print(f' {fault_type_n}')

#         # initialize result df
#         cv_error_df = pd.DataFrame([])

#         # self-defined fault types
#         selected_fault_types = [fault_type_n]    

# #         print(f'Processing: {weather}...')

#         # read metadata file
#         prefixed = [filename for filename in os.listdir(f'data/{weather}/{weather}/') if 'sensors' not in filename]
#         if len(prefixed) == 1:
#             meta_data_df = pd.read_csv(f'data/{weather}/{weather}/' + prefixed[0])
#         else:
#             raise Exception("Something wrong with finding the meta data file in the data folder. Make sure the original data folder are used for data processing")

#         # define the fault_type_list. Baseline is removed from the fault type
#         #     fault_type_list = meta_data_df.fault_type.unique().tolist()
#         #     fault_type_list.remove('baseline')
#         fault_type_list = selected_fault_types

#         inputs = pd.DataFrame([])
#         output = pd.DataFrame([])

#         for fault_type in fault_type_list:

#             #print(f' Processing: {fault_type}...')

#             # first focus on one weather file and one fault type
#             ids = meta_data_df.loc[meta_data_df.fault_type == fault_type].id.tolist()
#             intensities = meta_data_df.loc[meta_data_df.fault_type == fault_type].fault_intensity.tolist()

#             # non-fault IDs
#             ids_none_fault = meta_data_df.loc[meta_data_df.fault_type == 'baseline'].id.tolist()

#             # load data and make it from 15minute interval to hourly/daily data
#             fault_data_df = pd.DataFrame([])
#             for id_n, intensity in zip(ids, intensities):
#                 temp_df = pd.read_csv(f'data/{weather}/{weather}/{id_n}_sensors.csv')
#                 temp_df = temp_df.iloc[:,1:]
#                 temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
#                 #temp_df.loc[:,'label'] = intensity
#                 temp_df.loc[:,'label'] = fault_type
#                 fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)

#             # dealing with non fault data
#             temp_df = pd.read_csv(f'data/{weather}/{weather}/{ids_none_fault[0]}_sensors.csv')
#             temp_df = temp_df.iloc[:,1:]
#             temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
#             temp_df.loc[:,'label'] = 'none'
#             fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)
#             fault_data_df = fault_data_df.reset_index(drop = True)

#             inputs = pd.concat([inputs, fault_data_df.iloc[:,0:-9]], axis = 0, ignore_index = True)
#             output = pd.concat([output, fault_data_df.iloc[:,-1]], axis = 0, ignore_index = True)

# #         print(f' Processing: inputs/output data ready for {weather} ...')

#         # cross-validation
#         cv = KFold(n_splits=5, shuffle=True, random_state = 42)

#         # Iterate through CV splits
#         # results = []
#         important_features = []
#         i = 0
#         for train_index, test_index in cv.split(inputs):
#             X_train, X_test = inputs.iloc[train_index], inputs.iloc[test_index]
#             y_train, y_test = output.iloc[train_index], output.iloc[test_index]
#             # Fit the model on training data
#             regr = RandomForestClassifier(n_estimators = 200, random_state=42)
#             regr.fit(X_train, y_train.iloc[:,0])
# #             # save model
# #             filename = f'model/saved_model_{i}_{weather}.sav'
# #             pickle.dump(regr, open(filename, 'wb'))
# #             i += 1
#             # feature importance
#             feature_importance_temp = pd.DataFrame([])
#             feature_importance_temp.loc[:,'sensor_name'] = inputs.columns
#             feature_importance_temp.loc[:,'importance'] = regr.feature_importances_
#             important_features += feature_importance_temp.sort_values(
#                 by=['importance'], ascending = False).sensor_name[0:40].tolist()
#             break
#         #     # Generate predictions on the test data and collect
#         #     y_test_predicted = regr.predict(X_test)
#         #     testing_error = zero_one_loss(y_test, y_test_predicted, normalize=True)
#         #     results += [testing_error]

#         # CV_error = sum(results)/len(results)
#         # cv_error_df.loc[:,weather + 'Baseline'] = [weather, 'all_sensors', 'all_inaccuracy', CV_error]
#         # important_features = list(set(important_features))

#         # Sensor precision degradation

#         print(f'  Processing: Degradation, {weather}...')
#         # important_features_reversed = important_features
#         # important_features_reversed.reverse()
#         # for sensor in important_features_reversed:
#         for sensor in important_features[0:10]:
#             print(f'   Processing {sensor}')
#             inaccuracy = 0
#         #     results = []
        
#             threshold = 2000
#             interval = 100
#             while inaccuracy < threshold:
#         #         i = 0
#                 inaccuracy += interval
#                 print(f'     Inaccuracy: {inaccuracy}')
#                 for train_index, test_index in cv.split(inputs):
#                     X_train, X_test = inputs.iloc[train_index].copy()[important_features], inputs.iloc[test_index].copy()[important_features]
#                     y_train, y_test = output.iloc[train_index].copy(), output.iloc[test_index].copy()
#                     # degradation
#         #             random_list = []
#         # #             print(X_train[sensor])
#         #             for j in range(len(X_train)):
#         #                 random_list += [random.uniform(- X_train.loc[:,sensor].mean() * inaccuracy, X_train.loc[:,sensor].mean() * inaccuracy)]
#         # #                 random_list += [random.uniform(-inaccuracy, inaccuracy)]
#                     random_list = [random.uniform(- X_train.loc[:,sensor].mean() * inaccuracy, X_train.loc[:,sensor].mean() * inaccuracy) for j in range(len(X_train))]
#                     X_train.loc[:,sensor] = X_train.loc[:,sensor] + random_list
#         #             X_train.loc[:,sensor] = random_list
#         #             X_train.loc[:,sensor] = 1
#         #             X_train.loc[:,sensor] = 0
#         #             print(X_train[sensor])
#         #             print(len(X_train.columns))
#         #             # load saved pickle file
#         #             filename = f'model/saved_model_{i}_{weather}.sav'
#         #             regr = pickle.load(open(filename, 'rb'))
#         #             i += 1
#                     # train a new model
#         #             print(X_train[sensor])
#                     regr = RandomForestClassifier(n_estimators = 200, random_state = 42)
#         #             print(X_train[sensor])
#         #             print(1)
#                     regr.fit(X_train, y_train.iloc[:,0])
#         #             print(1)
#                     # Generate predictions on the test data and collect
#         #             y_test_predicted = regr.predict(X_test)
#         #             testing_error = zero_one_loss(y_test, y_test_predicted, normalize=True)
#         #             results += [testing_error]
#                     feature_importance_temp = pd.DataFrame([])
#                     feature_importance_temp.loc[:,'sensor_name'] = X_train.columns
#         #             print(len(X_train.columns))
#                     feature_importance_temp.loc[:,'importance'] = regr.feature_importances_
#         #             print(feature_importance_temp.sort_values(by=['importance'], ascending = False).sensor_name[0:10].tolist())
#                     important_features_degradation = []
#                     important_features_degradation += feature_importance_temp.sort_values(by=['importance'], ascending = False).sensor_name[0:10].tolist()
#                     break

#         #         important_features_degradation = list(set(important_features_degradation))
#                 if sensor not in important_features_degradation:
#                     print(f'      Threshold for {sensor} is {inaccuracy}')
#                     final_list.append([weather, selected_fault_types[0], sensor, inaccuracy])
#                     break
#                 else:
#                     if important_features_degradation.index(sensor) <= important_features.index(sensor):
#                         if inaccuracy >= threshold:
#                             final_list.append([weather, selected_fault_types[0], sensor, inaccuracy])
#                             continue
#                         else:
#                             continue
#                     else:
#                         print(f'      Threshold for {sensor} is {inaccuracy}')
#                         final_list.append([weather, selected_fault_types[0], sensor, inaccuracy])
#                         break

#         #     CV_error = sum(results)/len(results)
#         #     cv_error_df.loc[:,weather + sensor + 'degradation'] = [weather, sensor, 'degradation', CV_error]

In [30]:
all_fault_types = ['air_handling_unit_fan_motor_degradation',
                        'biased_economizer_sensor_mixed_t',
                        'duct_fouling',
                        'economizer_opening_stuck',
                        'hvac_setback_error_delayed_onset',
                        'hvac_setback_error_no_overnight_setback',
                        'hvac_setback_error_early_termination',
                        'improper_time_delay_setting_in_occupancy_sensors',
                        'lighting_setback_error_delayed_onset',
                        'lighting_setback_error_no_overnight_setback',
                        'lighting_setback_error_early_termination',
                        'return_air_duct_leakages',
                        'supply_air_duct_leakages',
                        'thermostat_bias',]

final_list = []

# for weather in ['AK_Fairbanks', 'FL_Miami', 'KY_Louisville', 'MN_Duluth', 'SAU_Riyadh', 'TN_Knoxville', 'VA_Richmond']:
for weather in ['SAU_Riyadh', 'TN_Knoxville', 'VA_Richmond']:
# for weather in ['AK_Fairbanks']:
    print(weather)

    for fault_type_n in all_fault_types:
        print(f' {fault_type_n}')

        # self-defined fault types
        selected_fault_types = [fault_type_n]    

        # read metadata file
        prefixed = [filename for filename in os.listdir(f'data/{weather}/{weather}/') if 'sensors' not in filename]
        if len(prefixed) == 1:
            meta_data_df = pd.read_csv(f'data/{weather}/{weather}/' + prefixed[0])
        else:
            raise Exception("Something wrong with finding the meta data file in the data folder. Make sure the original data folder are used for data processing")

        # define the fault_type_list. Baseline is removed from the fault type
        fault_type_list = selected_fault_types

        inputs = pd.DataFrame([])
        output = pd.DataFrame([])

        for fault_type in fault_type_list:

            #print(f' Processing: {fault_type}...')

            # first focus on one weather file and one fault type
            ids = meta_data_df.loc[meta_data_df.fault_type == fault_type].id.tolist()
            intensities = meta_data_df.loc[meta_data_df.fault_type == fault_type].fault_intensity.tolist()

            # non-fault IDs
            if weather == 'SAU_Riyadh':
                ids_none_fault = [meta_data_df.iloc[31,:].id]
            else:
                ids_none_fault = meta_data_df.loc[meta_data_df.fault_type == 'baseline'].id.tolist()

            # load data and make it from 15minute interval to hourly/daily data
            fault_data_df = pd.DataFrame([])
            for id_n, intensity in zip(ids, intensities):
                temp_df = pd.read_csv(f'data/{weather}/{weather}/{id_n}_sensors.csv')
                temp_df = temp_df.iloc[:,1:]
                temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
                #temp_df.loc[:,'label'] = intensity
                temp_df.loc[:,'label'] = fault_type
                fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)

            # dealing with non fault data
            temp_df = pd.read_csv(f'data/{weather}/{weather}/{ids_none_fault[0]}_sensors.csv')
            temp_df = temp_df.iloc[:,1:]
            temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
            temp_df.loc[:,'label'] = 'none'
            fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)
            fault_data_df = fault_data_df.reset_index(drop = True)

            inputs = pd.concat([inputs, fault_data_df.iloc[:,0:-9]], axis = 0, ignore_index = True)
            output = pd.concat([output, fault_data_df.iloc[:,-1]], axis = 0, ignore_index = True)

        # cross-validation
        cv = KFold(n_splits=5, shuffle=True, random_state = 42)

        # Iterate through CV splits
        for train_index, test_index in cv.split(inputs):
            X_train, X_test = inputs.iloc[train_index], inputs.iloc[test_index]
            y_train, y_test = output.iloc[train_index], output.iloc[test_index]
            # Fit the model on training data
            regr_1 = GradientBoostingClassifier(random_state = 42)
#             regr_1 = RandomForestClassifier(n_estimators = 500, random_state = 42)
#             regr_1 = DecisionTreeClassifier(random_state = 42)
            regr_1.fit(X_train, y_train.iloc[:,0])
            # feature importance
            feature_importance_temp = pd.DataFrame([])
            feature_importance_temp.loc[:,'sensor_name'] = inputs.columns
            feature_importance_temp.loc[:,'importance'] = regr_1.feature_importances_
            important_features = []
            important_features += feature_importance_temp.sort_values(
                by=['importance'], ascending = False).sensor_name[0:10].tolist() 
            break

        testing_error_init = 0
        for train_index, test_index in cv.split(inputs):
            # calculate original error
            y_test_predicted = regr_1.predict(X_test)
            testing_error_init += zero_one_loss(y_test, y_test_predicted, normalize=True)
        testing_error_init = testing_error_init / 5
        
        # Sensor precision degradation
        print(f'  Processing: Degradation, {weather}...')
        
        for sensor in important_features[0:10]:
            print(f'   Processing {sensor}')
            inaccuracy = 0
            threshold = 1 #* 10E8
            interval = 0.05 #* 10E8
            testing_error_list = [testing_error_init]
            while inaccuracy < threshold:
                inaccuracy += interval
                print(f'     Inaccuracy: {inaccuracy}')
                
                for train_index, test_index in cv.split(inputs):
#                     X_train, X_test = inputs.iloc[train_index].copy()[important_features], inputs.iloc[test_index].copy()[important_features]
                    X_train, X_test = inputs.iloc[train_index].copy(), inputs.iloc[test_index].copy()
                    y_train, y_test = output.iloc[train_index].copy(), output.iloc[test_index].copy()
                    random_list = [random.uniform(- X_train.loc[:,sensor].mean() * inaccuracy, X_train.loc[:,sensor].mean() * inaccuracy) for j in range(len(X_train))]
                    X_train.loc[:,sensor] = X_train.loc[:,sensor] + random_list

                    random_list = [random.uniform(- X_test.loc[:,sensor].mean() * inaccuracy, X_test.loc[:,sensor].mean() * inaccuracy) for j in range(len(X_test))]
                    X_test.loc[:,sensor] = X_test.loc[:,sensor] + random_list
                
                    regr_2 = GradientBoostingClassifier(random_state = 22)
#                     regr_2 = RandomForestClassifier(n_estimators = 500, random_state = 22)
#                     regr_2 = DecisionTreeClassifier(random_state = 32)
                    regr_2.fit(X_train, y_train.iloc[:,0])
    
                    y_test_predicted = regr_2.predict(X_test)
                    testing_error = zero_one_loss(y_test, y_test_predicted, normalize=True)
            
                    testing_error_list.append(testing_error)

                    feature_importance_temp_2 = pd.DataFrame([])
                    feature_importance_temp_2.loc[:,'sensor_name'] = X_train.columns
                    feature_importance_temp_2.loc[:,'importance'] = regr_2.feature_importances_
                    important_features_degradation = []
#                     important_features_degradation += feature_importance_temp_2.sort_values(by=['importance'], ascending = False).sensor_name[0:10].tolist()
                    important_features_degradation += feature_importance_temp_2.sort_values(by=['importance'], ascending = False).sensor_name.tolist()
                    break

#                 if sensor not in important_features_degradation:
                if important_features_degradation.index(sensor) > important_features.index(sensor):
                    print(f'      Threshold for {sensor} is {inaccuracy}')
                    final_list.append([weather, selected_fault_types[0], sensor, inaccuracy, testing_error_list])
                    break
#                 else:
#                     if important_features_degradation.index(sensor) <= important_features.index(sensor):
#                         if inaccuracy >= threshold:
#                             final_list.append([weather, selected_fault_types[0], sensor, inaccuracy])
# #                             continue
#                             break
#                         else:
#                             continue
#                     else:
#                         print(f'      Threshold for {sensor} is {inaccuracy}')
#                         final_list.append([weather, selected_fault_types[0], sensor, inaccuracy])
#                         break
                else:
                    continue

result_df = pd.DataFrame(final_list, columns = ['weather','fault_type','sensor','threshold','FDD_performance_list'])
result_df.to_csv('D:/jupyternotebook/FDD_sensor_impact_framework_applied_to_FRP/results/module_3/module_3_summarized_results_by_fault_types_rest_three_weather.csv')

SAU_Riyadh
 air_handling_unit_fan_motor_degradation
  Processing: Degradation, SAU_Riyadh...
   Processing fans_electricity [W]
     Inaccuracy: 0.05
      Threshold for fans_electricity [W] is 0.05
   Processing rooftop_supply_fan_fan_electric_energy [W]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
      Threshold for rooftop_supply_fan_fan_electric_energy [W] is 0.15000000000000002
   Processing rooftop_heating_coil_outlet_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
      Threshold for rooftop_heating_coil_outlet_system_node_current_density_volume_flow_rate [m3/s] is 0.1
   Processing rooftop_cooling_coil_outlet_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
      Threshold for rooftop_cooling_coil_outlet_system_node_current_density_volume_flow_rate [m3/s] is 0.15000000000000002
   Processing rooftop_cooling_coil_ou

     Inaccuracy: 0.7500000000000001
     Inaccuracy: 0.8000000000000002
     Inaccuracy: 0.8500000000000002
     Inaccuracy: 0.9000000000000002
     Inaccuracy: 0.9500000000000003
     Inaccuracy: 1.0000000000000002
   Processing environment_site_outdoor_air_drybulb_temperature [C]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
     Inaccuracy: 0.39999999999999997
     Inaccuracy: 0.44999999999999996
     Inaccuracy: 0.49999999999999994
     Inaccuracy: 0.5499999999999999
     Inaccuracy: 0.6
     Inaccuracy: 0.65
     Inaccuracy: 0.7000000000000001
     Inaccuracy: 0.7500000000000001
     Inaccuracy: 0.8000000000000002
     Inaccuracy: 0.8500000000000002
     Inaccuracy: 0.9000000000000002
     Inaccuracy: 0.9500000000000003
     Inaccuracy: 1.0000000000000002
   Processing node_6_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccurac

      Threshold for room_203_supply_inlet_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing node_6_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for node_6_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing room_202_supply_inlet_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for room_202_supply_inlet_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing node_18_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for node_18_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing {ced936e0-0614-4c8a-a4d6-aa0b5c362790}_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for {ced936e0-0614-4c8a-a4d6-aa0b5c362790}_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing interiorlights_electricity [W]
     Inaccuracy: 0.05
     In

     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
     Inaccuracy: 0.39999999999999997
     Inaccuracy: 0.44999999999999996
      Threshold for whole_building_facility_total_hvac_electric_demand_power [W] is 0.44999999999999996
   Processing room_103_supply_inlet_system_node_temperature [C]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
     Inaccuracy: 0.39999999999999997
     Inaccuracy: 0.44999999999999996
     Inaccuracy: 0.49999999999999994
     Inaccuracy: 0.5499999999999999
     Inaccuracy: 0.6
     Inaccuracy: 0.65
     Inaccuracy: 0.7000000000000001
     Inaccuracy: 0.7500000000000001
     Inaccuracy: 0.8000000000000002
     Inaccuracy: 0.8500000000000002
     Inaccuracy: 0.9000000000000002
     Inaccuracy: 0.9500000000000003
     Inaccuracy: 1.0000000000000002
   Pro

     Inaccuracy: 0.65
     Inaccuracy: 0.7000000000000001
     Inaccuracy: 0.7500000000000001
     Inaccuracy: 0.8000000000000002
     Inaccuracy: 0.8500000000000002
     Inaccuracy: 0.9000000000000002
     Inaccuracy: 0.9500000000000003
     Inaccuracy: 1.0000000000000002
   Processing room_203_supply_inlet_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
     Inaccuracy: 0.39999999999999997
     Inaccuracy: 0.44999999999999996
     Inaccuracy: 0.49999999999999994
     Inaccuracy: 0.5499999999999999
     Inaccuracy: 0.6
     Inaccuracy: 0.65
     Inaccuracy: 0.7000000000000001
     Inaccuracy: 0.7500000000000001
     Inaccuracy: 0.8000000000000002
     Inaccuracy: 0.8500000000000002
     Inaccuracy: 0.9000000000000002
     Inaccuracy: 0.9500000000000003
     Inaccuracy: 1.0000000000000002
   Processing node_9_system_no

...

     Inaccuracy: 0.1
      Threshold for node_12_system_node_current_density_volume_flow_rate [m3/s] is 0.1
   Processing node_19_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
      Threshold for node_19_system_node_current_density_volume_flow_rate [m3/s] is 0.35
   Processing node_9_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
     Inaccuracy: 0.39999999999999997
     Inaccuracy: 0.44999999999999996
      Threshold for node_9_system_node_current_density_volume_flow_rate [m3/s] is 0.44999999999999996
   Processing node_18_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.1500000000000

      Threshold for node_12_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing rooftop_supply_fan_fan_electric_energy [W]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
      Threshold for rooftop_supply_fan_fan_electric_energy [W] is 0.1
   Processing room_106_vav_reheat_damper_outlet_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for room_106_vav_reheat_damper_outlet_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing node_17_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for node_17_system_node_current_density_volume_flow_rate [m3/s] is 0.05
 lighting_setback_error_no_overnight_setback
  Processing: Degradation, VA_Richmond...
   Processing interiorlights_electricity [W]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
     Inaccuracy: 0.39999

     Inaccuracy: 0.1
      Threshold for {d5737138-bc5b-4212-acf2-fd3235458210}_system_node_current_density_volume_flow_rate [m3/s] is 0.1
   Processing node_24_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for node_24_system_node_current_density_volume_flow_rate [m3/s] is 0.05
   Processing room_105_supply_inlet_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
      Threshold for room_105_supply_inlet_system_node_current_density_volume_flow_rate [m3/s] is 0.15000000000000002
   Processing {76064095-69fd-4710-a706-3caca6b1930a}_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
      Threshold for {76064095-69fd-4710-a706-3caca6b1930a}_system_node_current_density_volume_flow_rate [m3/s] is 0.1
   Processing node_9_system_node_current_density_volume_flow_rate [m3/s]
     Inaccuracy: 0.05
      Threshold for node_9_syst

      Threshold for room_105_supply_inlet_system_node_temperature [C] is 0.05
   Processing room_104_supply_inlet_system_node_temperature [C]
     Inaccuracy: 0.05
      Threshold for room_104_supply_inlet_system_node_temperature [C] is 0.05
   Processing node_17_system_node_temperature [C]
     Inaccuracy: 0.05
      Threshold for node_17_system_node_temperature [C] is 0.05
   Processing room_205_supply_inlet_system_node_temperature [C]
     Inaccuracy: 0.05
      Threshold for room_205_supply_inlet_system_node_temperature [C] is 0.05
   Processing rooftop_cooling_coil_outlet_system_node_temperature [C]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
      Threshold for rooftop_cooling_coil_outlet_system_node_temperature [C] is 0.1
   Processing room_102_supply_inlet_system_node_temperature [C]
     Inaccuracy: 0.05
      Threshold for room_102_supply_inlet_system_node_temperature [C] is 0.05
   Processing room_202_zone_mean_air_temperature [C]
     Inaccuracy: 0.05
      Threshold for roo

FileNotFoundError: [Errno 2] No such file or directory: '/results/module_3/module_3_summarized_results_by_fault_types_rest_three_weather.csv'

In [7]:
result_df = pd.DataFrame(final_list, columns = ['weather','fault_type','sensor','threshold','FDD_performance_list'])
result_df.to_csv('module_3_summarized_results_by_fault_types.csv')

In [3]:
all_fault_types = ['air_handling_unit_fan_motor_degradation',
                        'biased_economizer_sensor_mixed_t',
                        'duct_fouling',
                        'economizer_opening_stuck',
                        'hvac_setback_error_delayed_onset',
                        'hvac_setback_error_no_overnight_setback',
                        'hvac_setback_error_early_termination',
                        'improper_time_delay_setting_in_occupancy_sensors',
                        'lighting_setback_error_delayed_onset',
                        'lighting_setback_error_no_overnight_setback',
                        'lighting_setback_error_early_termination',
                        'return_air_duct_leakages',
                        'supply_air_duct_leakages',
                        'thermostat_bias',]

final_list = []

#for weather in ['AK_Fairbanks', 'FL_Miami', 'KY_Louisville', 'MN_Duluth', 'SAU_Riyadh', 'TN_Knoxville', 'VA_Richmond']:
for weather in ['SAU_Riyadh', 'TN_Knoxville', 'VA_Richmond']:
# for weather in ['AK_Fairbanks']:
    print(weather)

    # read metadata file
    prefixed = [filename for filename in os.listdir(f'data/{weather}/{weather}/') if 'sensors' not in filename]
    if len(prefixed) == 1:
        meta_data_df = pd.read_csv(f'data/{weather}/{weather}/' + prefixed[0])
    else:
        raise Exception("Something wrong with finding the meta data file in the data folder. Make sure the original data folder are used for data processing")

    # define the fault_type_list. Baseline is removed from the fault type
    fault_type_list = all_fault_types

    inputs = pd.DataFrame([])
    output = pd.DataFrame([])

    for fault_type in fault_type_list:

        #print(f' Processing: {fault_type}...')

        # first focus on one weather file and one fault type
        ids = meta_data_df.loc[meta_data_df.fault_type == fault_type].id.tolist()
        intensities = meta_data_df.loc[meta_data_df.fault_type == fault_type].fault_intensity.tolist()

        # non-fault IDs
        if weather == 'SAU_Riyadh':
            ids_none_fault = ids_none_fault = [meta_data_df.iloc[31,:].id]
        else:
            ids_none_fault = meta_data_df.loc[meta_data_df.fault_type == 'baseline'].id.tolist()

        # load data and make it from 15minute interval to hourly/daily data
        fault_data_df = pd.DataFrame([])
        for id_n, intensity in zip(ids, intensities):
            temp_df = pd.read_csv(f'data/{weather}/{weather}/{id_n}_sensors.csv')
            temp_df = temp_df.iloc[:,1:]
            temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
            #temp_df.loc[:,'label'] = intensity
            temp_df.loc[:,'label'] = fault_type
            fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)

        # dealing with non fault data
        temp_df = pd.read_csv(f'data/{weather}/{weather}/{ids_none_fault[0]}_sensors.csv')
        temp_df = temp_df.iloc[:,1:]
        temp_df = temp_df.groupby(np.arange(len(temp_df))//(4*24)).mean()
        temp_df.loc[:,'label'] = 'none'
        fault_data_df = pd.concat([fault_data_df, temp_df], axis = 0)
        fault_data_df = fault_data_df.reset_index(drop = True)

        inputs = pd.concat([inputs, fault_data_df.iloc[:,0:-9]], axis = 0, ignore_index = True)
        output = pd.concat([output, fault_data_df.iloc[:,-1]], axis = 0, ignore_index = True)

    # cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state = 42)

    # Iterate through CV splits
    for train_index, test_index in cv.split(inputs):
        X_train, X_test = inputs.iloc[train_index], inputs.iloc[test_index]
        y_train, y_test = output.iloc[train_index], output.iloc[test_index]
        # Fit the model on training data
        regr_1 = GradientBoostingClassifier(random_state = 42)
#             regr_1 = RandomForestClassifier(n_estimators = 500, random_state = 42)
#             regr_1 = DecisionTreeClassifier(random_state = 42)
        regr_1.fit(X_train, y_train.iloc[:,0])
        # feature importance
        feature_importance_temp = pd.DataFrame([])
        feature_importance_temp.loc[:,'sensor_name'] = inputs.columns
        feature_importance_temp.loc[:,'importance'] = regr_1.feature_importances_
        important_features = []
        important_features += feature_importance_temp.sort_values(
            by=['importance'], ascending = False).sensor_name[0:10].tolist() 
        break

    testing_error_init = 0
    for train_index, test_index in cv.split(inputs):
        # calculate original error
        y_test_predicted = regr_1.predict(X_test)
        testing_error_init += zero_one_loss(y_test, y_test_predicted, normalize=True)
    testing_error_init = testing_error_init / 5

    # Sensor precision degradation
    print(f'  Processing: Degradation, {weather}...')

    for sensor in important_features[0:10]:
        print(f'   Processing {sensor}')
        inaccuracy = 0
        threshold = 1 #* 10E8
        interval = 0.05 #* 10E8
        testing_error_list = [testing_error_init]
        while inaccuracy < threshold:
            inaccuracy += interval
            print(f'     Inaccuracy: {inaccuracy}')

            for train_index, test_index in cv.split(inputs):
#                     X_train, X_test = inputs.iloc[train_index].copy()[important_features], inputs.iloc[test_index].copy()[important_features]
                X_train, X_test = inputs.iloc[train_index].copy(), inputs.iloc[test_index].copy()
                y_train, y_test = output.iloc[train_index].copy(), output.iloc[test_index].copy()
                random_list = [random.uniform(- X_train.loc[:,sensor].mean() * inaccuracy, X_train.loc[:,sensor].mean() * inaccuracy) for j in range(len(X_train))]
                X_train.loc[:,sensor] = X_train.loc[:,sensor] + random_list

                random_list = [random.uniform(- X_test.loc[:,sensor].mean() * inaccuracy, X_test.loc[:,sensor].mean() * inaccuracy) for j in range(len(X_test))]
                X_test.loc[:,sensor] = X_test.loc[:,sensor] + random_list

                regr_2 = GradientBoostingClassifier(random_state = 22)
#                     regr_2 = RandomForestClassifier(n_estimators = 500, random_state = 22)
#                     regr_2 = DecisionTreeClassifier(random_state = 32)
                regr_2.fit(X_train, y_train.iloc[:,0])

                y_test_predicted = regr_2.predict(X_test)
                testing_error = zero_one_loss(y_test, y_test_predicted, normalize=True)

                testing_error_list.append(testing_error)

                feature_importance_temp_2 = pd.DataFrame([])
                feature_importance_temp_2.loc[:,'sensor_name'] = X_train.columns
                feature_importance_temp_2.loc[:,'importance'] = regr_2.feature_importances_
                important_features_degradation = []
#                     important_features_degradation += feature_importance_temp_2.sort_values(by=['importance'], ascending = False).sensor_name[0:10].tolist()
                important_features_degradation += feature_importance_temp_2.sort_values(by=['importance'], ascending = False).sensor_name.tolist()
                break

#                 if sensor not in important_features_degradation:
            if important_features_degradation.index(sensor) > important_features.index(sensor):
                print(f'      Threshold for {sensor} is {inaccuracy}')
                final_list.append([weather, sensor, inaccuracy, testing_error_list])
                break
#                 else:
#                     if important_features_degradation.index(sensor) <= important_features.index(sensor):
#                         if inaccuracy >= threshold:
#                             final_list.append([weather, selected_fault_types[0], sensor, inaccuracy])
# #                             continue
#                             break
#                         else:
#                             continue
#                     else:
#                         print(f'      Threshold for {sensor} is {inaccuracy}')
#                         final_list.append([weather, selected_fault_types[0], sensor, inaccuracy])
#                         break
            else:
                continue

result_df = pd.DataFrame(final_list, columns = ['weather','sensor','threshold','FDD_performance_list'])
result_df.to_csv('D:/jupyternotebook/FDD_sensor_impact_framework_applied_to_FRP/results/module_3/module_3_summarized_results_rest_three_weather.csv')

SAU_Riyadh
  Processing: Degradation, SAU_Riyadh...
   Processing interiorlights_electricity [W]
     Inaccuracy: 0.05
     Inaccuracy: 0.1
     Inaccuracy: 0.15000000000000002
     Inaccuracy: 0.2
     Inaccuracy: 0.25
     Inaccuracy: 0.3
     Inaccuracy: 0.35
     Inaccuracy: 0.39999999999999997
     Inaccuracy: 0.44999999999999996
     Inaccuracy: 0.49999999999999994
     Inaccuracy: 0.5499999999999999
     Inaccuracy: 0.6
     Inaccuracy: 0.65
     Inaccuracy: 0.7000000000000001
     Inaccuracy: 0.7500000000000001
      Threshold for interiorlights_electricity [W] is 0.7500000000000001
   Processing rooftop_heating_coil_outlet_system_node_temperature [C]
     Inaccuracy: 0.05
      Threshold for rooftop_heating_coil_outlet_system_node_temperature [C] is 0.05
   Processing fans_electricity [W]
     Inaccuracy: 0.05
      Threshold for fans_electricity [W] is 0.05
   Processing rooftop_supply_fan_fan_electric_energy [W]
     Inaccuracy: 0.05
      Threshold for rooftop_supply_fan_fa

In [8]:
meta_data_df.fault_type.unique()

array(['duct_fouling', 'non_standard_charging_2',
       'lighting_setback_error_no_overnight_setback',
       'biased_economizer_sensor_outdoor_rh', 'condenser_fouling',
       'biased_economizer_sensor_oat', 'economizer_opening_stuck',
       'oversized_equipment_at_design', 'presence_of_non_condensable',
       'air_handling_unit_fan_motor_degradation',
       'biased_economizer_sensor_mixed_t', 'thermostat_bias',
       'condenser_fan_degradation',
       'hvac_setback_error_early_termination', 'liquid_line_restriction',
       'return_air_duct_leakages', 'hvac_setback_error_delayed_onset',
       'biased_economizer_sensor_return_t',
       'hvac_setback_error_no_overnight_setback',
       'biased_economizer_sensor_return_rh', nan,
       'supply_air_duct_leakages',
       'improper_time_delay_setting_in_occupancy_sensors',
       'lighting_setback_error_delayed_onset',
       'lighting_setback_error_early_termination',
       'excessive_infiltration'], dtype=object)

In [9]:
result_df = pd.DataFrame(final_list, columns = ['weather','fault_type','sensor','threshold','FDD_performance_list'])
result_df.to_csv('module_3_summarized_results.csv')